# Pymaceuticals Inc.
---

### Analysis

- Add your analysis here.
 

### Setup
------

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st

In [4]:
#Import mouse_metadata file
mouse_metadata_path = "data/Mouse_metadata.csv"
mouse_metadata_df = pd.read_csv(mouse_metadata_path)
mouse_metadata_df.head(1)

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16


In [5]:
#Import Study_data file
study_results_path = "data/Study_results.csv"
study_results_df = pd.read_csv(study_results_path)
study_results_df.head(1)

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0


In [7]:
# Combine the data into a single DataFrame
merged_results_df=pd.merge(mouse_metadata_df,study_results_df, on = ["Mouse ID"], how = "inner")


In [8]:
# Display the data table for preview
merged_results_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [14]:
# Checking for missing data
merged_results_df.count()

Mouse ID              1893
Drug Regimen          1893
Sex                   1893
Age_months            1893
Weight (g)            1893
Timepoint             1893
Tumor Volume (mm3)    1893
Metastatic Sites      1893
dtype: int64

In [25]:
# Checking the number of mice.
mouse_ids = merged_results_df["Mouse ID"].unique()
mouse_num = len(mouse_ids)
print (f'There are {mouse_num} mice.')

There are 249 mice.


In [33]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint.
# Optional: Get all the data for the duplicate mouse ID.

duplicate_etries = merged_results_df[merged_results_df.duplicated(['Mouse ID', 'Timepoint'])]
duplicate_etries 

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
909,g989,Propriva,Female,21,26,0,45.000000,0
911,g989,Propriva,Female,21,26,5,47.570392,0
913,g989,Propriva,Female,21,26,10,49.880528,0
915,g989,Propriva,Female,21,26,15,53.442020,0
917,g989,Propriva,Female,21,26,20,54.657650,1


In [34]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
cln_merged_results_df=merged_results_df.loc[merged_results_df["Mouse ID"] != "g989"]
# Display the dataframe
cln_merged_results_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [35]:
# Checking the number of mice in the clean DataFrame.
mouse_ids = cln_merged_results_df["Mouse ID"].unique()
mouse_num = len(mouse_ids)
print (f'There are {mouse_num} mice.')

There are 248 mice.


## Summary Statistics

In [51]:
# Create dataframe of tumor volumes grouped by drug regimen for analysis
groupby_regimen_tumor_volue_df = cln_merged_results_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"]

In [52]:
# Generate dataframe of mean tumor volumes by regimen
mean_tumor_volume_df = pd.DataFrame(groupby_regimen_tumor_volue_df.mean())
mean_tumor_volume_df = mean_tumor_volume_df.rename(columns={"Tumor Volume (mm3)":"Mean Tumor Volume (mm3)"})
mean_tumor_volume_df.head(1)

,Mean Tumor Volume (mm3)
Drug Regimen,
Capomulin,40.675741


In [53]:
# Generate dataframe of median tumor volumes by regimen
median_tumor_volume_df = pd.DataFrame(groupby_regimen_tumor_volue_df.median())
median_tumor_volume_df = median_tumor_volume_df.rename(columns={"Tumor Volume (mm3)":"Median Tumor Volume (mm3)"})
median_tumor_volume_df.head(1)

,Median Tumor Volume (mm3)
Drug Regimen,
Capomulin,41.557809


In [55]:
# Generate dataframe of standard deviations of tumor volumes by regimen
var_tumor_volume_df = pd.DataFrame(groupby_regimen_tumor_volue_df.var(ddof=0))
var_tumor_volume_df = var_tumor_volume_df.rename(columns={"Tumor Volume (mm3)":"Variance of Tumor Volume (mm3)"})
var_tumor_volume_df.head(1)

,Variance of Tumor Volume (mm3)
Drug Regimen,
Capomulin,24.839296


In [58]:
# Generate dataframe of standard deviations of tumor volumes by regimen
std_dev_tumor_volume_df = pd.DataFrame(groupby_regimen_tumor_volue_df.std(ddof=0))
std_dev_tumor_volume_df = std_dev_tumor_volume_df.rename(columns={"Tumor Volume (mm3)":"Standard Deviation of Tumor Volume (mm3)"})
std_dev_tumor_volume_df.head(1)

,Standard Deviation of Tumor Volume (mm3)
Drug Regimen,
Capomulin,4.983904


In [61]:
# Generate dataframe of standard error of tumor volumes by regimen
sem_tumor_volume_df = pd.DataFrame(groupby_regimen_tumor_volue_df.sem())
sem_tumor_volume_df = sem_tumor_volume_df.rename(columns={"Tumor Volume (mm3)":"Standard Error of Tumor Volume (mm3)"})
sem_tumor_volume_df.head(1)

,Standard Error of Tumor Volume (mm3)
Drug Regimen,
Capomulin,0.329346


In [66]:
# Assemble the resulting dataframes into a single summary DataFrame
regimen_tumor_volue_summary_df = mean_tumor_volume_df
regimen_tumor_volue_summary_df=pd.merge(regimen_tumor_volue_summary_df,median_tumor_volume_df, on = ["Drug Regimen"], how = "inner")
regimen_tumor_volue_summary_df=pd.merge(regimen_tumor_volue_summary_df,var_tumor_volume_df, on = ["Drug Regimen"], how = "inner")
regimen_tumor_volue_summary_df=pd.merge(regimen_tumor_volue_summary_df,std_dev_tumor_volume_df, on = ["Drug Regimen"], how = "inner")
regimen_tumor_volue_summary_df=pd.merge(regimen_tumor_volue_summary_df,sem_tumor_volume_df, on = ["Drug Regimen"], how = "inner")
regimen_tumor_volue_summary_df

,Mean Tumor Volume (mm3),Median Tumor Volume (mm3),Variance of Tumor Volume (mm3),Standard Deviation of Tumor Volume (mm3),Standard Error of Tumor Volume (mm3)
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.839296,4.983904,0.329346
Ceftamin,52.591172,51.776157,39.069446,6.250556,0.469821
Infubinol,52.884795,51.820584,42.886388,6.548770,0.492236
Ketapril,55.235638,53.698743,68.188930,8.257659,0.603860
Naftisol,54.331565,52.509285,65.817708,8.112811,0.596466
Placebo,54.033581,52.288934,60.830138,7.799368,0.581331
Propriva,52.320930,50.446266,43.555716,6.599675,0.544332
Ramicane,40.216745,40.673236,23.383692,4.835669,0.320955
Stelasyn,54.233149,52.431737,59.122106,7.689090,0.573111


In [ ]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.


In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot


## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes

    
    # add subset 

    
    # Determine outliers using upper and lower bounds


In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.


## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and a linear regression model 
# for mouse weight and average observed tumor volume for the entire Capomulin regimen
